# 手把手教你写 Transformer (Step-by-Step)

欢迎！本教程将带你从零开始，一行一行地构建一个 GPT 风格的 Transformer 模型。

我们将从 Transformer 的灵魂 —— **Scaled Dot-Product Attention (缩放点积注意力)** 开始。

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

# 设置随机种子以保证结果可复现
torch.manual_seed(1337)

print("环境准备就绪！")

环境准备就绪！


## 第一步：理解核心公式

Attention 的核心公式如下：

$$ \text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V $$

- **Q (Query)**: 查询向量 (比如我手里拿着的 token)
- **K (Key)**: 键向量 (比如被查询的 token)
- **V (Value)**: 值向量 (如果不相关，我就不关注你；如果相关，我从你这里提取什么信息)

让我们先用随机生成的 Tensor 来模拟这个过程，不涉及任何神经网络层。

In [2]:
# 1. 定义数据的维度
B = 1  # Batch size (批次大小)
T = 8  # Time steps (序列长度，比如 "I love deep learning" 是 4 个 token，这里我们设为 8)
C = 32 # Channels (embedding size，每个 token 用多少维向量表示)
head_size = 16 # 单个注意力头的维度 (d_k)

# 2. 生成随机的 Q, K, V
# 在真实模型中，这些是由 input x 经过 nn.Linear 投影得到的
key   = torch.randn(B, T, head_size)
query = torch.randn(B, T, head_size)
value = torch.randn(B, T, head_size)

print(f"Q shape: {query.shape}")
print(f"K shape: {key.shape}")
print(f"V shape: {value.shape}")

Q shape: torch.Size([1, 8, 16])
K shape: torch.Size([1, 8, 16])
V shape: torch.Size([1, 8, 16])


### 📝 练习 1: 原始注意力计算

请尝试补全下面的代码，实现上述公式。

**提示**：
1. 矩阵乘法 $QK^T$: 使用 `@` 运算符或 `torch.matmul`。注意 `K` 需要转置最后两个维度。
2. 转置操作: `k.transpose(-2, -1)`
3. 缩放: 除以 `head_size` 的平方根。
4. Softmax: `F.softmax(..., dim=-1)`
5. 加权求和: 注意力权重矩阵乘以 `V`。

In [4]:
def raw_attention(q, k, v):
    d_k = q.shape[-1] 
    
    # --- TODO: 请在下面补全代码 ---
    
    # 1. 计算 Attention Scores (weights) = Q @ K^T
    # weights shape 应该是 (B, T, T)
    # weights = ...
    weights = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(d_k)
    
    # 2. 缩放 (Scale)
    # weights = ...
    
    # 3. 归一化 (Softmax)
    # weights = ...
    weights = torch.softmax(weights,dim=-1)
    # 4. 加权聚合 (Aggregate) = weights @ V
    # out = ...
    out = weights @ V
    # --- 代码结束 ---
    
    return out, weights

# 运行测试（取消下面一行的注释来测试）
out, weights = raw_attention(query, key, value)
print("Output shape:", out.shape) # 应该是 (1, 8, 16)
print("Weights shape:", weights.shape) # 应该是 (1, 8, 8)

NameError: name 'V' is not defined

## 第二步：加入因果掩码 (Causal Masking)

对于 GPT (Decoder-only) 模型，**Token 不能看到未来的信息**。例如，生成第 5 个字的时候，只能看第 1,2,3,4,5 个字，不能偷看第 6 个字。

我们需要在 Softmax 之前，将矩阵的上三角部分设置为负无穷大 (`-inf`)。

### 📝 练习 2: 实现 Causal Attention

In [ ]:
def causal_attention(q, k, v):
    d_k = q.shape[-1]
    T = q.shape[1] #..

    
    # 1. 计算 Q @ K^T 并缩放
    weights = q @ k.transpose(-2, -1) * (d_k ** -0.5)
    
    # --- TODO: 实现 Masking ---
    # 创建一个下三角矩阵 (tril)，其中 1 代表可见，0 代表不可见
    # 使用 torch.tril 和 torch.ones
    # mask = ...
    
    # 使用 masked_fill 将 mask 为 0 的位置填充为 float('-inf')
    # weights = weights.masked_fill(...)
    
    # --- 代码结束 ---
    
    weights = F.softmax(weights, dim=-1)
    out = weights @ v
    return out, weights

# 运行测试
# out, wei = causal_attention(query, key, value)
# print("Output shape:", out.shape)
# print("Weights sample (row 0):", wei[0, 0, :]) # 应该只能看到第一个元素，其他接近 0

## 第三步：封装成 nn.Module

现在我们将逻辑封装到一个 PyTorch 类中。这次我们会加入 `nn.Linear` 来生成 Q, K, V。

In [ ]:
class SingleHeadAttention(nn.Module):
    def __init__(self, n_embd, head_size, block_size):
        super().__init__()
        self.head_size = head_size
        
        # 定义三个线性层
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        
        # 注册 mask 不作为模型参数更新
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

    def forward(self, x):
        # x 的维度: (Batch, Time-step, Channels)
        B, T, C = x.shape
        
        # --- TODO: 完成前向传播 ---
        # 1. 生成 k, q, v (通过 self.key(x) 等)
        
        # 2. 计算 attention scores (带缩放)
        
        # 3. Apply mask (使用 self.tril[:T, :T])
        
        # 4. Softmax
        
        # 5. Aggregate output
        
        out = None # 修改这里
        
        return out

# 测试 Module
model = SingleHeadAttention(n_embd=32, head_size=16, block_size=8)
x = torch.randn(1, 8, 32)
out = model(x)
print("Module Output shape:", out.shape) # (1, 8, 16)